In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import geopy.distance


%store -r words_count 
%store -r host_attributes_1
%store -r host_attributes_2
%store -r review_1
%store -r review_2
%store -r house_attributes
%store -r policies
%store -r location_

### read in the data and replace all missing with 0

In [3]:
## we didnt get rid of duplicated listings because after adding monthly fixed effect
## we treat every point as an individual data(i.e some same listing improved listing attributes over time)
df_listings = pd.read_csv('listing_1_yr_beijing.csv',index_col=0)
df_listings['year'] = df_listings['year'].apply(lambda x: x[:7])


time_dummy = pd.get_dummies(df_listings['year']).iloc[:,1:] ## add monthly fixed effect, drop 2018-10 as baseline
df_listings = pd.concat([df_listings,time_dummy], axis = 1)
df_listings = df_listings.replace({'bedrooms': np.nan, 'beds': np.nan,'bathrooms': np.nan}, 0)  ## fill all missing values with 0
#df_listings = df_listings.replace(np.nan,'0')

## got rid of min_night more than 28, since otherwise they are rented house advertisement
df_listings = df_listings.loc[df_listings.minimum_nights <= 28, :] 
## gor rid of beds more than 10 beds, most of them doesn exist
df_listings = df_listings.loc[df_listings.beds <= 10, :] 
## gor rid of beds more than 10 bedroomss, most of them doesn exist
df_listings = df_listings.loc[df_listings.bedrooms <= 10, :] 
## gor rid of beds more than 10 bathrooms, most of them doesn exist
df_listings = df_listings.loc[df_listings.bathrooms <= 10, :] 

### covert price to euro 

In [4]:
df_listings['price'] = df_listings.price.apply(lambda x: str(x)[1:-3].replace(',',''))
df_listings['price in Euro'] = df_listings.price.apply(lambda x: int(x) * 0.13)

### generate info count

In [5]:
#remove price<=0
df_listings= df_listings.loc[df_listings['price in Euro']>0, :] 

In [6]:
## generate a info column decribe how many words are there in the descriptional info 
## such as summary, space, description
df_listings['info_count'] = 0

for word in words_count:
    count = df_listings[word].apply(lambda x: 0 if str(x) == '0' else len(str(x).split(' ')))
    df_listings['info_count'] += count

df_listings = df_listings.drop(words_count,axis=1)

### clean the data for house_attributes 

In [7]:
df_listings = df_listings.replace({'bedrooms': 0, 'beds': 0}, 1)

In [8]:
# there are 4 differenct room type ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']
room_type_dummy = pd.get_dummies(df_listings['room_type']).iloc[:,1:] ## dropped entire apt as baseline
df_listings = pd.concat([df_listings,room_type_dummy], axis = 1)

# property_type -- divide into special, non special and missing values
property_type = [x for x in df_listings.property_type.unique()]

property_type_special = ['Condominium','Houseboat','Boat','Other','Tiny house','House',\
                         'Townhouse','Cottage', 'Nature lodge', 'Igloo', 'Cave','Earth house',\
                         'Treehouse', 'Resort', 'Campsite', 'Castle','In-law', 'Chalet', 'Tipi', \
                         'Hut', 'Island', 'Lighthouse','Vacation home', 'Farm stay', 'Timeshare',\
                         'Earth House', 'Casa particular', 'Train', 'Floor']

property_type_non_special= list(set(property_type) - set(property_type_special))

df_listings['property_type'] = df_listings['property_type'].apply(lambda x: 0 if x in property_type_special else x)
df_listings['property_type'] = df_listings['property_type'].apply(lambda x: 1 if x in property_type_non_special else x)




# amenities_counts -- the number of amenities the listing provides
df_listings['amenities_counts'] = df_listings['amenities'].apply(lambda x: 0 if str(x) == 0 else len(str(x).split(',')))

### location model variables

In [9]:
df_listings['cordinates'] = list(zip(df_listings.latitude, df_listings.longitude))
cord_1=(39.9055,116.3976)
df_listings['Tiananmen'] = df_listings.cordinates.apply(lambda x: geopy.distance.distance(x, cord_1).km)
cord_2=(40.0000,116.2755)
df_listings['Summerpalace'] = df_listings.cordinates.apply(lambda x: geopy.distance.distance(x, cord_2).km)
cord_3=(39.8822,116.4066)
df_listings['HavenPalce'] = df_listings.cordinates.apply(lambda x: geopy.distance.distance(x, cord_3).km)
cord_4=(40.0081,116.2982)
df_listings['Yuanmingyuan'] = df_listings.cordinates.apply(lambda x: geopy.distance.distance(x, cord_4).km)
cord_5=(39.9924,116.3394)
df_listings['Wudaokou'] = df_listings.cordinates.apply(lambda x: geopy.distance.distance(x, cord_5).km)

df_listings['distance_Km'] = df_listings[['Tiananmen', 'Summerpalace','HavenPalce','Yuanmingyuan','Wudaokou']].mean(axis=1)

df_listings = df_listings.drop(['cordinates','Tiananmen', 'Summerpalace','HavenPalce','Yuanmingyuan','Wudaokou'],axis = 1)

### policies

In [10]:
## instant_bookable is a binary variable
df_listings['instant_bookable'] = df_listings['instant_bookable'].map({'f':0,'t':1})

# policy_dummy['flexible', ['strict_14_with_grace_period', 'moderate',
#              'super_strict_30', 'strict', 'super_strict_60',
#             'luxury_super_strict_95']
## group only to strict and nonstrict

df_listings.loc[df_listings.cancellation_policy != 'flexible', 'cancellation_policy'] = 1
df_listings.loc[df_listings.cancellation_policy == 'flexible', 'cancellation_policy'] = 0

### reviews 

In [17]:
##drop missing values according to the info of reviews

df_listings = df_listings.dropna(axis=0, subset=['review_scores_rating','review_scores_accuracy','review_scores_cleanliness',\
                          'review_scores_checkin','review_scores_communication','review_scores_location',\
                          'review_scores_value'])

In [12]:
df_listings= df_listings.loc[df_listings['review_scores_rating']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_accuracy']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_cleanliness']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_checkin']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_communication']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_location']>0, :] 
df_listings= df_listings.loc[df_listings['review_scores_value']>0, :] 

In [13]:
df_listings['review_scores_rating']= df_listings.review_scores_rating.apply(lambda x: int(x)/20)
df_listings['review_scores_accuracy']= df_listings.review_scores_accuracy.apply(lambda x: int(x)/2)
df_listings['review_scores_cleanliness']= df_listings.review_scores_cleanliness.apply(lambda x: int(x)/2)
df_listings['review_scores_checkin']= df_listings.review_scores_checkin.apply(lambda x: int(x)/2)
df_listings['review_scores_communication']= df_listings.review_scores_communication.apply(lambda x: int(x)/2)
df_listings['review_scores_location']= df_listings.review_scores_location.apply(lambda x: int(x)/2)
df_listings['review_scores_value']= df_listings.review_scores_value.apply(lambda x: int(x)/2)

In [14]:
df_listings['if_reviewed'] = df_listings.number_of_reviews.apply(lambda x: 0 if x == '0' else 1)

### host_attributes 

In [15]:
#superhost is a binary variable 1=t and f=0
df_listings['host_is_superhost'] = df_listings['host_is_superhost'].map({'f':0,'t':1})
#host identity verified 1=yes and f=0 binary variable some of the superhosts are not verified there for this variables is necessary
df_listings['host_identity_verified'] = df_listings['host_identity_verified'].map({'f':0,'t':1})

In [16]:
df_listings.shape

(249557, 52)

### export csv file

In [19]:
df_listings.to_csv('airbnbdata_cleaned.csv')

In [20]:
%store time_dummy
%store room_type_dummy

Stored 'time_dummy' (DataFrame)
Stored 'room_type_dummy' (DataFrame)
